# Responsible AI: XAI GenAI project

## 0. Background



Based on the previous lessons on explainability, post-hoc methods are used to explain the model, such as saliency map, SmoothGrad, LRP, LIME, and SHAP. Take LRP (Layer Wise Relevance Propagation) as an example; it highlights the most relevant pixels to obtain a prediction of the class "cat" by backpropagating the relevance. (image source: [Montavon et. al (2016)](https://giorgiomorales.github.io/Layer-wise-Relevance-Propagation-in-Pytorch/))

<!-- %%[markdown] -->
![LRP example](images/catLRP.jpg)

Another example is about text sentiment classification, here we show a case of visualizing the importance of words given the prediction of 'positive':

![text example](images/textGradL2.png)

where the words highlight with darker colours indicate to be more critical in predicting the sentence to be 'positive' in sentiment.
More examples could be found [here](http://34.160.227.66/?models=sst2-tiny&dataset=sst_dev&hidden_modules=Explanations_Attention&layout=default).

Both cases above require the class or the prediction of the model. But:

***How do you explain a model that does not predict but generates?***

In this project, we will work on explaining the generative model based on the dependency between words. We will first look at a simple example, and using Point-wise Mutual Information (PMI) to compute the saliency map of the sentence. After that we will contruct the expereiment step by step, followed by exercises and questions.


## 1. A simple example to start with
Given a sample sentence: 
> *Tokyo is the capital city of Japan.* 

We are going to explain this sentence by finding the dependency using a saliency map between words.
The dependency of two words in the sentence could be measured by [Point-wise mutual information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information): 


Mask two words out, e.g. 
> \[MASK-1\] is the captial city of \[MASK-2\].


Ask the generative model to fill in the sentence 10 times, and we have:

| MASK-1      | MASK-2 |
| ----------- | ----------- |
|    tokyo   |     japan   |
|  paris  |     france    |
|  london  |     england    |
|  paris  |     france    |
|  beijing |  china |
|    tokyo   |     japan   |
|  paris  |     france    |
|  paris  |     france    |
|  london  |     england    |
|  beijing |  china |

PMI is calculated by: 

$PMI(x,y)=log_2⁡ \frac{p(\{x,y\}| s-\{x,y\})}{P(\{x\}|s-\{x,y\})P(\{y\}|s-\{x,y\})}$

where $x$, $y$ represents the words that we masked out, $s$ represents the setence, and $s-\{x,y\}$ represents the sentences tokens after removing the words $x$ and $y$.

In this example we have $PMI(Tokyo, capital) = log_2 \frac{0.2}{0.2 * 0.2} = 2.32$

Select an interesting word in the sentences; we can now compute the PMI between all other words and the chosen word using the generative model:
(Here, we use a longer sentence and run 20 responses per word.)
![](images/resPMI.png)


## 2. Preparation
### 2.1 Conda enviroment

```
conda env create -f environment.yml
conda activate xai_llm
```


### 2.2 Download the offline LLM

We use the offline LLM model from hugging face. It's approximately 5 GB.
Download it using the comman below, and save it under `./models/`.
```
huggingface-cli download TheBloke/openchat-3.5-0106-GGUF openchat-3.5-0106.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# credit to https://huggingface.co/TheBloke/openchat-3.5-0106-GGUF
```

## 3. Mask the sentence and get the responses from LLM
### 3.1 Get the input sentence

**Remember to change the anchor word index when changing the input sentence.**

In [13]:
def get_input():
    # ideally this reads inputs from a file, now it just takes an input
    return input("Enter a sentence: ")
    
anchor_word_idx = 0 # the index of the interested word
prompts_per_word = 20 # number of generated responses  

sentence ="Tokyo is the capital of Japan"
sentence = sentence.lower()
print("Sentence: ", sentence)

Sentence:  tokyo is the capital of japan


### 3.2 Load the model

In [2]:
from models.ChatModel import ChatModel
model_name = "openchat"
model = ChatModel(model_name)
print(f"Model: {model_name}")

Model: openchat


### 3.3 Run the prompts and get all the responses


In [3]:
from tools.command_generator import generate_prompts, prefix_prompt
from tools.evaluate_response import get_replacements
from tqdm import tqdm

def run_prompts(model, sentence, anchor_idx, prompts_per_word=20):
    prompts = generate_prompts(sentence, anchor_idx)
    all_replacements = []
    for prompt in prompts:
        replacements = []
        for _ in tqdm(
            range(prompts_per_word),
            desc=f"Input: {prompt}",
        ):
            response = model.get_response(
                prefix_prompt(prompt),
            ).strip()
            if response:
                replacement = tuple(get_replacements(prompt, response))
                if replacement:
                    replacements.append(replacement)
        if len(replacements) > 0:
            all_replacements.append(replacements)
    return all_replacements

all_responses = run_prompts(model, sentence, anchor_word_idx, prompts_per_word)

Input: [MASK] is [MASK] capital of Japan:  55%|█████▌    | 11/20 [00:10<00:07,  1.19it/s]

 Response is not valid. ['[mask]', 'is', '[mask]', 'capital', 'of', 'japan'] ['tokyo', 'is', 'japans', 'capital']


Input: [MASK] is the capital [MASK] Japan:   5%|▌         | 1/20 [00:00<00:12,  1.50it/s]

 Response is not valid. ['[mask]', 'is', 'the', 'capital', '[mask]', 'japan'] ['print']


Input: [MASK] is the capital [MASK] Japan:  80%|████████  | 16/20 [00:11<00:02,  1.73it/s]

...

Input: [MASK] is the capital [MASK] Japan:  95%|█████████▌| 19/20 [00:13<00:00,  1.35it/s]

 Response is not valid. ['[mask]', 'is', 'the', 'capital', '[mask]', 'japan'] ['tokyo', 'is', 'the', 'capital', '[tokyo]']


Input: [MASK] is the capital of [MASK]:   5%|▌         | 1/20 [00:01<00:28,  1.53s/it]

 Response is not valid. ['[mask]', 'is', 'the', 'capital', 'of', '[mask]'] ['须弥是日本的首都']


Input: [MASK] is the capital of [MASK]: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


### 3.4 EXERCISE: compute the PMI for each word

$PMI(x,y)=log_2⁡ \frac{p(\{x,y\}| s-\{x,y\})}{P(\{x\}|s-\{x,y\})P(\{y\}|s-\{x,y\})}$

* Compute the $P(x)$, $P(y)$ and $P(x,y)$ first and print it out.
* Compute the PMI for each word.
* Visualize the result by coloring. Tips: you might need to normalize the result first. 


In [4]:
for i, replacements in enumerate(all_responses):
    print(f"Prompt: {generate_prompts(sentence, anchor_word_idx)[i]}")
    for replacement in replacements:
        print(replacement)
    print("\n")

Prompt: [MASK] [MASK] the capital of Japan
('\x0e', 'tokyo is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('the', 'city of tokyo is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')
('tokyo', 'is')


Prompt: [MASK] is [MASK] capital of Japan
('chrome', 'not the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'tokyos')
('tokyo', 'the')
('', '')
('tokyo', 'the')
('tokyo', 'tokyos')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')
('tokyo', 'the')


Prompt: [MASK] is the [MASK] of Japan
('opyright', 'cultural property')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'capital')
('tokyo', 'c

In [ ]:
import numpy as np



def caluclate_pmi(sentence, anchor_word_idx, all_responses):
    anchor_word = sentence.split()[anchor_word_idx]
    pmi_dict = {}
    
    i = 0
    for word in sentence.split():
        if word != anchor_word:
            frequency_of_pairs = Counter(all_responses[i])
            frequency_of_X = Counter([pair[0] for pair in all_responses[i]])
            frequency_of_Y = Counter([pair[1] for pair in all_responses[i]])
            
            p_x = frequency_of_X[anchor_word] / len(all_responses[i])
            p_y = frequency_of_Y[word] / len(all_responses[i])
            p_xy = frequency_of_pairs[(anchor_word, word)] / len(all_responses[i])
            epsilon = 1e-10
            pmi = np.log2(p_xy / (p_x * p_y + epsilon))
            pmi_dict[word] = pmi
            i += 1
    return pmi_dict


import matplotlib.pyplot as plt

def visualize_pmi(pmi_dict):
    words = list(pmi_dict.keys())
    pmi_values = np.array(list(pmi_dict.values()))
    
    # Normalize the PMI values
    norm = plt.Normalize(pmi_values.min(), pmi_values.max())
    norm_pmi_values = norm(pmi_values)
    
    # Create a color map
    cmap = plt.get_cmap('coolwarm')
    colors = cmap(norm_pmi_values)
    
    # Plot the words with colors based on PMI values
    plt.figure(figsize=(10, 1))
    for i, word in enumerate(words):
        plt.text(i, 0.5, word, fontsize=12, ha='center', va='center', color=colors[i])
    
    plt.xlim(-1, len(words))
    plt.axis('off')
    plt.show()

pmi_dict = caluclate_pmi(sentence, anchor_word_idx, all_responses)
visualize_pmi(pmi_dict)

{'is': np.float64(0.15200309326693948),
 'the': np.float64(0.15200309324467542),
 'capital': np.float64(0.234465253437342),
 'of': np.float64(0.23446525341071792),
 'japan': np.float64(2.3219280912806246)}


## 4. EXERCISE: Try more examples; maybe come up with your own. Report the results.

* Try to come up with more examples and, change the anchor word/number of responses, and observe the results. What does the explanation mean? Do you think it's a nice explanation? Why and why not? 
* What's the limitation of the current method? When does the method fail to explain? 

## 5. Bonus Exercises
### 5.1 Language pre-processing. 
In this exercise, we only lower the letters and split sentences into words; there's much more to do to pre-process the language. For example, contractions (*I'll*, *She's*, *world's*), suffix and prefix, compound words (*hard-working*). It's called word tokenization in NLP, and there are some Python packages that can do such work for us, e.g. [*TextBlob*](https://textblob.readthedocs.io/en/dev/). 


### 5.2 Better word matching
In the above example of
> Tokyo is the capital of Japan and a popular metropolis in the world.

, GenAI never gives the specific word 'metropolis' when masking it out; instead, sometimes it provides words like 'city', which is not the same word but has a similar meaning. Instead of measuring the exact matching of certain words (i.e. 0 or 1), we can also measure the similarity of two words, e.g. the cosine similarity in word embedding, which ranges from 0 to 1. 